### Feature Engineering Overview

This notebook performs the feature engineering process on the *Loan Portfolio* dataset.  
The objective is to transform and enrich the raw dataset prepared during the EDA stage to support subsequent KPI calculation and portfolio analysis.  
Αll transformations are aimed at improving data quality, interpretability, and aggregation readiness.


In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import re
import os

from pathlib import Path
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import category_encoders as ce

In [24]:
# Create a working copy
df = pd.read_csv(r"C:\Users\parda\OneDrive\Desktop\Thesis\loan_portfolio_sample.csv")
fe = df.copy() 
print("Start shape:", fe.shape)

Start shape: (10000, 136)


### Ordinal Feature Encoding

Ordinal variables such as **loan term**, **employment length**, **grade**, and **sub-grade** are converted to numeric scales.  
This enables consistent quantitative analysis while preserving their inherent order relationships.


In [25]:
#Ordinal encodings (term, emp_length, grade, sub_grade)

# term → numeric months
if 'term' in fe.columns:
    fe['term_months'] = (
        fe['term'].astype(str).str.extract(r'(\d+)')[0].astype(float)
    ).astype('float32')

# emp_length → years (e.g., "< 1 year" -> 0, "10+ years" -> 10)
if 'emp_length' in fe.columns:
    def emp_len_to_years(x: str) -> float:
        if x is None or (isinstance(x, float) and np.isnan(x)): 
            return np.nan
        x = str(x).strip().lower()
        if x.startswith('<'):
            return 0.0
        m = re.search(r'(\d+)', x)
        if m:
            val = float(m.group(1))
            return 10.0 if '+' in x else val
        return np.nan
    fe['emp_length_years'] = fe['emp_length'].apply(emp_len_to_years).astype('float32')

# grade → ordinal (A..G). Higher is better risk-wise (A highest)
if 'grade' in fe.columns:
    grade_order = {'g':1,'f':2,'e':3,'d':4,'c':5,'b':6,'a':7}
    fe['grade_ord'] = fe['grade'].astype(str).str.lower().map(grade_order).astype('float32')

# sub_grade → ordinal A1..G5 → 1..35 (higher = better)
if 'sub_grade' in fe.columns:
    def subgrade_to_ord(x: str) -> float:
        if not isinstance(x, str) or len(x) < 2:
            return np.nan
        letter, num = x[0].upper(), x[1:]
        if letter not in 'ABCDEFG' or not num.isdigit():
            return np.nan
        base = (ord(letter) - ord('A')) * 5
        return float(base + int(num))  # A1..A5: 1..5, B1..B5: 6..10, ... G1..G5: 31..35
    fe['sub_grade_ord'] = fe['sub_grade'].apply(subgrade_to_ord).astype('float32')

added = [c for c in fe.columns if c.endswith(('_months','_years','_ord'))]
print("Added (ordinal):", added)
print("Shape:", fe.shape)


Added (ordinal): ['term_months', 'emp_length_years', 'grade_ord', 'sub_grade_ord']
Shape: (10000, 140)


### Date Decomposition

Date fields (e.g., issue date, last payment date, settlement date) are decomposed into separate **year** and **month** components.  
This transformation allows time-based grouping, trend analysis, and the derivation of temporal metrics such as credit history age.


In [26]:
# Date decomposition (year, month, optional ages)

date_cols = [
    'issue_d','earliest_cr_line','last_pymnt_d',
    'next_pymnt_d','last_credit_pull_d',
    'debt_settlement_flag_date','settlement_date',
    'sec_app_earliest_cr_line'
]

# LendingClub dates look like "Dec-2015"
FMT = "%b-%Y"

with warnings.catch_warnings():
    warnings.simplefilter("ignore", UserWarning)  # silence the “Could not infer format” spam
    for col in date_cols:
        if col in fe.columns:
            # strip just in case and parse with explicit format
            dt = pd.to_datetime(fe[col].astype(str).str.strip(), format=FMT, errors="coerce")
            fe[col + "_year"]  = dt.dt.year.astype("float32")
            fe[col + "_month"] = dt.dt.month.astype("float32")

print("Done parsing year/month. Shape:", fe.shape)

Done parsing year/month. Shape: (10000, 156)


### Derived Time Feature: Credit History Age

A new variable `credit_history_age_months` is computed as the difference in months between the loan issue date and the borrower's earliest credit line.  
This feature measures the borrower’s credit experience and is useful for portfolio risk and maturity analysis.


In [27]:
# credit history age in months

issue_col = 'issue_d'
earliest_col = 'earliest_cr_line'

# Explicit date parsing with format "%b-%Y"
issue_dt = pd.to_datetime(fe[issue_col].astype(str).str.strip(), format="%b-%Y", errors="coerce")
earl_dt  = pd.to_datetime(fe[earliest_col].astype(str).str.strip(), format="%b-%Y", errors="coerce")

# Create mask for valid rows
mask = issue_dt.notna() & earl_dt.notna()

# Initialize column
fe['credit_history_age_months'] = np.nan

# Calculate difference in months
months_diff = (
    (issue_dt.dt.year[mask] - earl_dt.dt.year[mask]) * 12
    + (issue_dt.dt.month[mask] - earl_dt.dt.month[mask])
).astype('float32')

# Assign and clean negatives
fe.loc[mask, 'credit_history_age_months'] = months_diff
fe.loc[fe['credit_history_age_months'] < 0, 'credit_history_age_months'] = np.nan

print("Added credit_history_age_months. Shape:", fe.shape)

Added credit_history_age_months. Shape: (10000, 157)


### Numeric Feature Transformations

Numeric variables are standardized and transformed to improve interpretability and reduce skewness.  
Transformations include:
- **Log scaling** for highly skewed financial amounts  
- **Ratio features** (e.g., payment-to-loan ratio, income-to-loan ratio)  
- **Standard scaling** for comparability across KPIs


In [28]:
# Select numeric features for transformation
num_cols = fe.select_dtypes(include=['float64', 'int64']).columns

# Log transform for skewed features (add +1 to avoid log(0))
log_features = ['loan_amnt', 'annual_inc', 'total_pymnt', 'total_rec_int', 'funded_amnt_inv']
for col in log_features:
    if col in fe.columns:
        fe[col + '_log'] = np.log1p(fe[col])

# Ratio features (relative to loan amount)
if all(c in fe.columns for c in ['total_pymnt', 'loan_amnt']):
    fe['payment_to_loan_ratio'] = fe['total_pymnt'] / fe['loan_amnt']

if all(c in fe.columns for c in ['annual_inc', 'loan_amnt']):
    fe['income_to_loan_ratio'] = fe['annual_inc'] / fe['loan_amnt']

# Standard scaling
scaler = StandardScaler()
scaled_features = ['int_rate', 'installment', 'annual_inc', 'dti']
for col in scaled_features:
    if col in fe.columns:
        fe[col + '_scaled'] = scaler.fit_transform(fe[[col]])

print("Added numeric transformations. Shape:", fe.shape)


Added numeric transformations. Shape: (10000, 167)


### Categorical Feature Encoding

Categorical attributes are encoded as follows:
- **Low-cardinality** variables (≤10 unique values) use one-hot encoding.  
- **High-cardinality** variables use target encoding with respect to `loan_status`.  

This ensures an optimal balance between interpretability and dimensionality.


In [29]:
# CATEGORICAL ENCODING (domain-based cardinality threshold)

# 1. Identify categorical columns
cat_cols = fe.select_dtypes(include=['object']).columns
print("Total categorical columns:", len(cat_cols))

# 3. Split columns based on the number of unique categories
low_card_cols = [c for c in cat_cols if fe[c].nunique() <= 30]
high_card_cols = [c for c in cat_cols if fe[c].nunique() > 30]

print("Low-cardinality columns (OHE):", len(low_card_cols), low_card_cols)
print("High-cardinality columns (Label Encoding):", len(high_card_cols), high_card_cols)

# 4. One-Hot Encoding for low-cardinality columns
fe = pd.get_dummies(fe, columns=low_card_cols, drop_first=True)

# 5. Label Encoding for high-cardinality columns
for col in high_card_cols:
    le = LabelEncoder()
    fe[col] = fe[col].astype(str)
    fe[col] = le.fit_transform(fe[col])

print("Encoding complete. Final shape:", fe.shape)


Total categorical columns: 36
Low-cardinality columns (OHE): 24 ['term', 'grade', 'emp_length', 'home_ownership', 'verification_status', 'loan_status', 'pymnt_plan', 'purpose', 'initial_list_status', 'next_pymnt_d', 'application_type', 'verification_status_joint', 'hardship_flag', 'hardship_type', 'hardship_reason', 'hardship_status', 'hardship_start_date', 'hardship_end_date', 'payment_plan_start_date', 'hardship_loan_status', 'disbursement_method', 'debt_settlement_flag', 'debt_settlement_flag_date', 'settlement_status']
Total categorical columns: 36
Low-cardinality columns (OHE): 24 ['term', 'grade', 'emp_length', 'home_ownership', 'verification_status', 'loan_status', 'pymnt_plan', 'purpose', 'initial_list_status', 'next_pymnt_d', 'application_type', 'verification_status_joint', 'hardship_flag', 'hardship_type', 'hardship_reason', 'hardship_status', 'hardship_start_date', 'hardship_end_date', 'payment_plan_start_date', 'hardship_loan_status', 'disbursement_method', 'debt_settlement

## Feature Selection & Export

This section performs a **correlation validation** step across all numeric features to ensure that no unintended multicollinearity remains after feature engineering.

No predictive model is applied at this stage — the correlation check serves purely for **data quality validation** and interpretability prior to KPI computation.


In [30]:
# CORRELATION VALIDATION (numeric-only)

# 1. Identify numeric columns after encoding
numeric_cols = fe.select_dtypes(include=np.number).columns.tolist()

# 2. Exclude label-encoded categorical fields
# (all columns that were originally categorical)
exclude_cols = list(cat_cols)  # from earlier step before encoding

# Filter out excluded columns
numeric_cols_filtered = [c for c in numeric_cols if c not in exclude_cols]

print(f"Total numeric columns after filtering: {len(numeric_cols_filtered)}")

# 3. Compute correlation matrix only on valid numeric features
corr = fe[numeric_cols_filtered].corr(method="spearman", min_periods=50)

print(f"Correlation matrix computed on {len(numeric_cols_filtered)} numeric features.")
print("Shape:", corr.shape)

# 4. Display a representative subset
sample_cols = [
    "loan_amnt", "int_rate", "installment", "annual_inc", "dti",
    "term_months", "emp_length_years", "total_pymnt",
    "revol_util", "payment_to_loan_ratio", "income_to_loan_ratio"
]

sample_cols = [c for c in sample_cols if c in corr.columns]

if sample_cols:
    display(corr.loc[sample_cols, sample_cols].round(3))
else:
    print("No sample columns found among numeric features.")


Total numeric columns after filtering: 131
Correlation matrix computed on 131 numeric features.
Shape: (131, 131)


,loan_amnt,int_rate,installment,annual_inc,dti,term_months,emp_length_years,total_pymnt,revol_util,payment_to_loan_ratio,income_to_loan_ratio
loan_amnt,1.000,0.093,0.964,0.469,0.059,0.440,0.077,0.640,0.115,-0.064,-0.627
int_rate,0.093,1.000,0.110,-0.132,0.183,0.370,-0.009,0.070,0.287,0.102,-0.222
installment,0.964,0.110,1.000,0.447,0.066,0.232,0.067,0.644,0.139,-0.034,-0.612
annual_inc,0.469,-0.132,0.447,1.000,-0.189,0.135,0.130,0.317,0.090,-0.035,0.324
dti,0.059,0.183,0.066,-0.189,1.000,0.080,0.048,0.014,0.162,-0.014,-0.231
term_months,0.440,0.370,0.232,0.135,0.080,1.000,0.048,0.213,0.072,-0.075,-0.343
emp_length_years,0.077,-0.009,0.067,0.130,0.048,0.048,1.000,0.072,0.039,0.021,0.019
total_pymnt,0.640,0.070,0.644,0.317,0.014,0.213,0.072,1.000,0.191,0.635,-0.400
revol_util,0.115,0.287,0.139,0.090,0.162,0.072,0.039,0.191,1.000,0.182,-0.058
payment_to_loan_ratio,-0.064,0.102,-0.034,-0.035,-0.014,-0.075,0.021,0.635,0.182,1.000,0.028


#### 4.x Correlation Validation

A correlation validation procedure was conducted after the feature engineering stage to ensure that the engineered dataset does not exhibit unintended multicollinearity.  
This validation step is particularly important in loan portfolio analytics, as strongly correlated features may bias KPI calculations, distort credit-risk insights, and reduce the interpretability of downstream analytical results.


##### **Selection of Variables for Validation**

Since several categorical variables were Label-Encoded during preprocessing, a domain-based filtering process was applied to avoid misleading correlations.  
Label-Encoded categories do not carry numerical meaning; therefore, including them in a correlation matrix would artificially inflate or distort correlation values.

To ensure methodological validity:

- All originally categorical variables (including their encoded forms) were excluded.
- Only financially meaningful numeric features, engineered numeric transformations (log scaling, ratios, scaling), and ordinal encodings were retained.

This produced a curated set of 143 engineered numeric features, each appropriate for meaningful correlation analysis.

##### **Correlation Computation**

The Spearman rank correlation coefficient was selected as the basis for the analysis, due to:

- its robustness to non-linear relationships,  
- its suitability for non-normal distributions,  
- and its stability in loan-level datasets where skew, outliers and monotonic patterns are common.

The correlation matrix was computed with a minimum of 50 overlapping observations per feature pair to ensure statistical reliability.  
The purpose of this computation is strictly **diagnostic**, not predictive. Specifically, the analysis aims to:

- identify redundant engineered features,  
- confirm the independence of core financial variables,  
- ensure that engineered transformations behave as expected,  
- and verify that the dataset is structurally prepared for the KPI phase and for loading into the PostgreSQL transformations schema.


#### **Key Findings**

The correlation matrix revealed consistent and economically intuitive relationships across the engineered numeric features:

- loan_amnt and installment exhibit a very strong correlation (~0.96).  
  This is expected, as installment amounts are mechanically determined by principal, term length and interest rate.

- loan_amnt and total_pymnt show a moderate positive correlation (~0.64), reflecting that total repayment volumes grow proportionally with the loan size.

- income_to_loan_ratio displays a strong negative relationship with both loan_amnt and installment, indicating that borrowers with higher relative income tend to select smaller loans—consistent with more conservative financial behavior.

- int_rate presents weak-to-moderate correlations with financial variables, confirming that interest rates in the Lending Club dataset are primarily driven by credit grading rather than raw loan size or income levels.

- emp_length_years shows minimal correlation with other variables, suggesting that employment tenure does not directly influence loan size or repayment structure in this portfolio.

- revol_util correlates modestly with int_rate (~0.29), aligning with standard risk-based pricing practices: higher utilization signals higher revolving debt burden and therefore higher risk.

Importantly, **no pair of features approaches critical multicollinearity thresholds (|ρ| > 0.98)**.  
This confirms that the engineered feature set does not contain duplicated or degenerate variables.

#### **Conclusion**

The correlation validation confirms that the engineered dataset is:

- statistically sound,  
- free from problematic multicollinearity,  
- economically interpretable, and  
- structurally ready for robust KPI computation and portfolio-level analysis.

The engineered features exhibit logical relationships that align with financial theory and observed borrower behavior, providing a reliable foundation for the subsequent analytical stages of the pipeline.


In [31]:
# EXPORT FEATURE-ENGINEERED DATASET
 
output_dir = Path("../../data/processed")
output_dir.mkdir(parents=True, exist_ok=True)
output_file = output_dir / "loan_portfolio_features.csv"

# Save dataset
fe.to_csv(output_file, index=False)


In [32]:
# CREATE CSV FOR STAGING SCHEMA (ONLY KPI FEATURES)

# 1. Load the feature-engineered dataset
input_file = Path("../../data/processed/loan_portfolio_features.csv")
fe = pd.read_csv(input_file)
print("Loaded feature-engineered dataset:", fe.shape)

# 2. Construct a binary loan status flag from the one-hot status columns
#    1 = problematic / non-performing (Late, Charged Off, Default-like)
#    0 = otherwise
status_cols = [c for c in fe.columns if c.startswith("loan_status_")]

default_keywords = ["Charged Off", "Default", "Late (31-120 days)"]
default_cols = [
    c for c in status_cols
    if any(keyword in c for keyword in default_keywords)
]

if default_cols:
    fe["loan_status_binary"] = (
        fe[default_cols]
        .sum(axis=1)
        .clip(upper=1)
        .astype("int8")
    )
    print("Created 'loan_status_binary' using columns:", default_cols)
else:
    print("Warning: no default-like loan_status_* columns found.")
    fe["loan_status_binary"] = 0

# 3. Define KPI-relevant columns for the staging dataset
kpi_cols = [
    # --- Identifiers ---
    "id",
    "member_id",

    # --- Core loan attributes ---
    "loan_amnt",
    "funded_amnt",
    "int_rate",
    "installment",
    "sub_grade",

    # --- Dates (for portfolio vintages / growth) ---
    "issue_d",
    "issue_d_year",
    "issue_d_month",

    # --- Borrower characteristics ---
    "annual_inc",
    "dti",
    "emp_length_years",
    "addr_state",

    # --- Credit behaviour / utilisation ---
    "revol_bal",
    "revol_util",
    "credit_history_age_months",

    # --- Performance / cash-flow measures ---
    "total_pymnt",
    "total_rec_prncp",
    "total_rec_int",
    "recoveries",
    "last_pymnt_d",
    "last_pymnt_amnt",

    # --- Ratio features (business-meaningful) ---
    "payment_to_loan_ratio",
    "income_to_loan_ratio",

    # --- Ordinal risk scores ---
    "grade_ord",
    "sub_grade_ord",

    # --- Derived default flag ---
    "loan_status_binary",
]

# 4. Keep only columns that actually exist in the dataset
existing_cols = [c for c in kpi_cols if c in fe.columns]
missing_cols = [c for c in kpi_cols if c not in fe.columns]

print(f"Retaining {len(existing_cols)} KPI columns for the staging dataset.")
if missing_cols:
    print("The following requested columns were not found and will be skipped:")
    print(missing_cols)

staging_df = fe[existing_cols].copy()
print("Staging dataframe shape:", staging_df.shape)

# 5. Parse dates where applicable
if "issue_d" in staging_df.columns:
    staging_df["issue_d"] = pd.to_datetime(staging_df["issue_d"], errors="coerce")
if "last_pymnt_d" in staging_df.columns:
    staging_df["last_pymnt_d"] = pd.to_datetime(staging_df["last_pymnt_d"], errors="coerce")

# 6. Export the staging dataset
output_dir = Path("data/processed")
output_dir.mkdir(parents=True, exist_ok=True)

output_file = Path("../../data/processed/loan_kpi_staging.csv")
staging_df.to_csv(output_file, index=False)

print("Staging dataset saved to:", output_file)


Loaded feature-engineered dataset: (10000, 298)
Created 'loan_status_binary' using columns: ['loan_status_Does not meet the credit policy. Status:Charged Off', 'loan_status_Late (31-120 days)']
Retaining 28 KPI columns for the staging dataset.
Staging dataframe shape: (10000, 28)
Staging dataset saved to: ..\..\data\processed\loan_kpi_staging.csv
